In [1]:
##select folder with data in it

# pandas, numpy, ipywidgets, matplotlib, and tkinker need to be installed to run this notebook
#tkinker only needed for this first data selection cell

import tkinter as tk
from tkinter import filedialog
from IPython.display import clear_output


#use this to navigate to select data folder
def select_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    folder_path = filedialog.askdirectory()
    root.destroy()  # Close the main window
    return folder_path

print("Select your folder:")
folder_path = select_folder()
clear_output()

print(f"Selected folder: {folder_path}")


Selected folder: /Users/adamhunter/Downloads/Project Documents-20230328/Data


In [2]:
##read in excel files into a df dictionary

import os
import pandas as pd

# Get the list of files in the selected folder
file_names = os.listdir(folder_path)

# Initialize an empty dictionary to store DataFrames
dfs = {}

# Read each Excel file into a DataFrame and add to the dictionary
# with the file name (without extension) as key
for file_name in file_names:
    if file_name.endswith('.xlsx'):
        # Get the full path of the file
        file_path = os.path.join(folder_path, file_name)
        
        # Extract file name without extension
        name = os.path.splitext(file_name)[0]
        
        # Read the Excel file into a DataFrame and store it in the dictionary
        dfs[name] = pd.read_excel(file_path)


In [3]:
##drop any rows that are exactly the same in all dfs

dfs['product_details'].drop_duplicates(inplace=True)
dfs['timeline_details'].drop_duplicates(inplace=True)
dfs['order_details'].drop_duplicates(inplace=True)
dfs['returns_list'].drop_duplicates(inplace=True)
dfs['customer_details'].drop_duplicates(inplace=True)

In [ ]:
##join all dfs together

# Load the order details data into a pandas DataFrame
order_details_df = dfs['order_details']

# Join the other DataFrames with the order details DataFrame
for key, df in dfs.items():
    if key != 'order_details':
        common_cols = list(set(df.columns) & set(order_details_df.columns))
        
        if len(common_cols) > 0:
            common_col = common_cols[0]
            print(f"Joining '{key}' DataFrame on '{common_col}' column")
            order_details_df = pd.merge(order_details_df, df, on=common_col, how='left')
        else:
            print(f"Cannot join '{key}' DataFrame as no common column found with order_details_df")

# Set the maximum number of columns to display to None
pd.set_option('display.max_columns', None)

# Drop duplicate data resulting from product and timeline joins
order_details_df = order_details_df.drop_duplicates(subset=['Row ID'])

# Display the final DataFrame
order_details_df


In [ ]:
##create dashboard

# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import clear_output
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from ipywidgets import Output


%matplotlib widget
%matplotlib inline

# Load DataFrame
data = order_details_df

df = pd.DataFrame(data)
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['delivery_datetime'] = pd.to_datetime(df['delivery_datetime'])
df['package_datetime'] = pd.to_datetime(df['package_datetime'])

# Create widgets
region_dropdown = widgets.Dropdown(
    options=['All'] + list(df['Region'].unique()),
    value='All',
    description='Region:',
)

start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=datetime.strptime('2021-01-01', '%Y-%m-%d')
)

end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=datetime.strptime('2022-01-01', '%Y-%m-%d')
)

# Filter DataFrame based on widget values
def filter_data(region, start_date, end_date):
    filtered_df = df.copy()

    if region != 'All':
        filtered_df = filtered_df[filtered_df['Region'] == region]

    # Convert date values to datetime64[ns] type
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)

    filtered_df = filtered_df[(filtered_df['Order Date'] >= start_date) & (filtered_df['Order Date'] <= end_date)]
    
    return filtered_df


def calculate_metrics(filtered_df):
    # Drop rows with missing values in relevant columns
    filtered_df = filtered_df.dropna(subset=['delivery_datetime', 'order_datetime', 'package_datetime'])

    # Calculate durations in hours
    filtered_df['delivery_duration'] = (filtered_df['delivery_datetime'] - filtered_df['order_datetime']).dt.total_seconds() / 3600
    filtered_df['package_duration'] = (filtered_df['package_datetime'] - filtered_df['order_datetime']).dt.total_seconds() / 3600
    
    avg_delivery_duration = filtered_df['delivery_duration'].mean()
    avg_package_duration = filtered_df['package_duration'].mean()
    
    sales_by_day = filtered_df.groupby('Order Date')['Sales'].sum()
    avg_sales_per_day = sales_by_day.mean()
    
    avg_shipping_cost = filtered_df['Shipping Cost per order'].mean()
    avg_discount = filtered_df['Discount per item'].mean()
    
    return avg_delivery_duration, avg_package_duration, avg_sales_per_day, avg_shipping_cost, avg_discount
    
def calculate_retention_rate(filtered_df, state=None):
    if state:
        filtered_df = filtered_df[filtered_df['State or Province'] == state]

    # Count the number of unique customers who have made repeat purchases
    repeat_customers = filtered_df[filtered_df.duplicated(subset='Customer ID', keep=False)]['Customer ID'].nunique()

    # Count the total number of unique customers
    total_customers = filtered_df['Customer ID'].nunique()

    # Calculate the retention rate
    retention_rate = repeat_customers / total_customers * 100

    return retention_rate

def retention_rate_and_sales_by_state(filtered_df):
    retention_rates = []
    sales_by_state = []
    states = filtered_df['State or Province'].unique()

    for state in states:
        state_data = filtered_df[filtered_df['State or Province'] == state]
        retention_rate = calculate_retention_rate(state_data, state)  # Pass the state parameter
        total_sales = state_data['Sales'].sum()

        retention_rates.append(retention_rate)
        sales_by_state.append(total_sales)

        
    return pd.DataFrame({'State': states, 'Retention Rate': retention_rates, 'Sales': sales_by_state})

def calc_correl(filtered_df):
# i want to calculate the correlation between sales and package duration by day
    sales_by_day = filtered_df.groupby('Order Date')['Sales'].sum()
    package_duration_by_day = filtered_df.groupby('Order Date')['package_duration'].mean()
    corr = sk.pearsonr(sales_by_day, package_duration_by_day)
    return corr


def display_widgets():
    display(region_dropdown)
    display(start_date_picker)
    display(end_date_picker)
    display(refresh_button)


def display_results():
    with output_widget:
        clear_output(wait=True)
    
        filtered_df = filter_data(region_dropdown.value, start_date_picker.value, end_date_picker.value)
        avg_delivery_duration, avg_package_duration, avg_sales_per_day, avg_shipping_cost, avg_discount = calculate_metrics(filtered_df)
        print(f"KPIs:")
        print(f"Average Time to Delivery: {avg_delivery_duration:.2f} hours")
        print(f"Average Time to Package: {avg_package_duration:.2f} hours")
        print(f"Average Sales per Day: ${avg_sales_per_day:.2f}")
        print(f"Average Shipping Cost Per Order: ${avg_shipping_cost:.2f}")
        print(f"Average Discount Rate: {avg_discount:.2f}")

        # Calculate retention rate
        retention_rate = calculate_retention_rate(filtered_df)
        print(f"Customer Retention Rate: {retention_rate:.2f}%")

        print(f"Charts:")
          # Line graph for total sales by day

        sales_by_day = filtered_df.groupby('Order Date')['Sales'].sum()
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.plot(sales_by_day.index, sales_by_day.values)
        plt.title('Total Sales by Day')
        plt.xlabel('Date (2021)')
        plt.ylabel('Sales')

        # Format y-axis as dollar
        formatter = mticker.FormatStrFormatter('$%1.0f')
        ax.yaxis.set_major_formatter(formatter)

        # Format x-axis to show month and day
        date_fmt = mdates.DateFormatter('%m-%d')
        ax.xaxis.set_major_formatter(date_fmt)

        plt.show()
        
        # Bar graph for sales by Product Category
        sales_by_category = filtered_df.groupby('Product Category')['Sales'].sum().sort_values(ascending=False)
        fig, ax = plt.subplots(figsize=(10, 5))
        bars = ax.bar(sales_by_category.index, sales_by_category.values)
        plt.title('Sales by Product Category')
        plt.xlabel('Product Category')
        plt.ylabel('Sales')

        # Format y-axis as dollar
        formatter = mticker.FormatStrFormatter('$%1.0f')
        ax.yaxis.set_major_formatter(formatter)

        # Add data labels on bars
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f"${height:.0f}",
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

        plt.show()

            # Bar graph for customer ID counts and total sales by Customer Segment
        customer_segment_data = filtered_df.groupby('Customer Segment').agg({'Customer ID': 'nunique', 'Sales': 'sum'})
        customer_segment_data = customer_segment_data.sort_values(by=['Customer ID'], ascending=False)

        fig, ax1 = plt.subplots(figsize=(10, 5))

        # Plot the bar graph for customer ID counts
        bar_width = 0.35
        index = np.arange(len(customer_segment_data.index))

        color = 'tab:blue'
        ax1.set_xlabel('Customer Segment')
        ax1.set_ylabel('Customer ID Counts', color=color)
        rects1 = ax1.bar(index, customer_segment_data['Customer ID'], bar_width, label='Customer Counts', color=color, alpha=0.5)
        ax1.tick_params(axis='y', labelcolor=color)

        # Set the y-axis limits for ax1
        ax1.set_ylim(0, customer_segment_data['Customer ID'].max() * 1.1)


        # Plot the bar graph for total sales on the same graph with a different y-axis
        ax2 = ax1.twinx()
        color = 'tab:orange'
        ax2.set_ylabel('Total Sales', color=color)
        rects2 = ax2.bar(index + bar_width, customer_segment_data['Sales'], bar_width, label='Total Sales', color=color, alpha=0.5)
        ax2.tick_params(axis='y', labelcolor=color)

        # Set the y-axis limits for ax2
        ax2.set_ylim(0, customer_segment_data['Sales'].max() * 1.1)


        # Format y-axis as dollar
        formatter = mticker.FormatStrFormatter('$%1.0f')
        ax2.yaxis.set_major_formatter(formatter)

        ax1.set_xticks(index + bar_width / 2)
        ax1.set_xticklabels(customer_segment_data.index)

        # Add a legend
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='upper right')

        # Add data labels
        def autolabel(rects, ax):
            for rect in rects:
                height = rect.get_height()
                ax.annotate(f"{height:.0f}",
                            xy=(rect.get_x() + rect.get_width() / 2, height),
                            xytext=(0, 3),
                            textcoords="offset points",
                            ha='center', va='bottom')

        autolabel(rects1, ax1)
        autolabel(rects2, ax2)

        plt.title('Customer Counts and Total Sales by Customer Segment')
        plt.show()

        
        
        customer_distribution = filtered_df.groupby('State or Province')['Customer ID'].nunique()
        fig, ax = plt.subplots(figsize=(10, 5))
        bars = ax.bar(customer_distribution.index, customer_distribution.values)
        plt.title('Customer Distribution by State')
        plt.xlabel('State')
        plt.ylabel('Number of Customers')
        
        plt.xticks(rotation=90)  # Rotate x-axis labels for readability

        plt.show()
        
        state_data = retention_rate_and_sales_by_state(filtered_df)

        fig, ax = plt.subplots(figsize=(10, 5))
        sc = ax.scatter(state_data['Retention Rate'], state_data['Sales'], alpha=0.5)

        plt.title('Customer Retention Rate vs. Sales by State')
        plt.xlabel('Retention Rate (%)')
        plt.ylabel('Sales')

        # Format y-axis as dollar
        formatter = mticker.FormatStrFormatter('$%1.0f')
        ax.yaxis.set_major_formatter(formatter)

        for i, state in enumerate(state_data['State']):
            ax.annotate(state, (state_data['Retention Rate'][i], state_data['Sales'][i]), fontsize=8, alpha=0.7)

        plt.show()

        

       



#display stuff
refresh_button = widgets.Button(description='Refresh Results')
refresh_button.on_click(lambda x: display_results())

output_widget = Output()
display_widgets()
display(output_widget)

